# **Computações com Python III**

In [51]:
from sympy import integrate, var, cos, sin, pi, Integral, plot, Abs, exp
x = var('x')
t = var('t')

### **O espaço de funções analíticas**

**Exercício 54.1**

In [52]:
## Função que retorna o produto interno entre duas funções:
def inner_product(f, g, var, a, b):
    inner_product = integrate(f*g, (var, a, b))
    return inner_product

In [53]:
## Testes:
print(inner_product( x, x**2, x, -1, 1 ))
print(inner_product( 1, x**2, x, -1, 1 ))
print(inner_product( cos(x), sin(x), x, -pi, pi))
print(inner_product( cos(x), cos(x), x, -pi, pi))

0
2/3
0
pi


**Exercício 54.2**

In [54]:
## Função que retorna a projeção ortogonal de uma função f sobre outra função g entre duas funções:
def orthogonal_projection(f, g, var, a, b):
    orthogonal_projection = (inner_product(f, g, var, a, b)/(inner_product(g, g, var, a, b)))*g
    return orthogonal_projection

In [55]:
## Testes: 
print(orthogonal_projection( x, x**2, x, -1, 1))
print(orthogonal_projection( 1, x**2, x, -1, 1))
print(orthogonal_projection( x**2, cos(x), x, -pi, pi))

0
5*x**2/3
-4*cos(x)


### **Ortogonalização de Gram-Schmidt**

**Exercício 54.3**

In [56]:
## Função aplica a ortogonalização de gram-schmidt a uma lista de funções:
def gram_schmidt(funcs, var, a, b):
    ws = []
    for f in funcs:
           ws.append(f - sum(orthogonal_projection(f, w, var, a, b) for w in ws)) ## Dica do Csaba
    return ws

In [57]:
## Testes:
print(gram_schmidt([1,x,x**2], x, -1, 1))
print(gram_schmidt([1,x,x**2], x, 0, 1))
print(gram_schmidt([1,cos(x),sin(x)], x, -pi, pi))

[1, x, x**2 - 1/3]
[1, x - 1/2, x**2 - x + 1/6]
[1, cos(x), sin(x)]


**Exercício 54.4**

In [58]:
R4_list = [1, x, x**2, x**3, x**4]

## Intervalo [-1,1]:
orthogonal_span = gram_schmidt(R4_list, x, -1, 1)
print(orthogonal_span)

## Teste para ver se a lista é ortogonal:
for fixed_f in orthogonal_span:
    for dinamic_f in orthogonal_span:
        if fixed_f == dinamic_f:
            break
        ip = inner_product( fixed_f, dinamic_f, x, -1, 1)
        if ip == 0:
            print("Ortogonal")
        else:
            print("null")

## Intervalo [0,1]:
orthogonal_span = gram_schmidt(R4_list, x, 0, 1)
print(orthogonal_span)

## Teste para ver se a lista é ortogonal:
for fixed_f in orthogonal_span:
    for dinamic_f in orthogonal_span:
        if fixed_f == dinamic_f:
            break
        ip = inner_product( fixed_f, dinamic_f, x, 0, 1)
        if ip == 0:
            print("Ortogonal")
        else:
            print("null")

[1, x, x**2 - 1/3, x**3 - 3*x/5, x**4 - 6*x**2/7 + 3/35]
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
[1, x - 1/2, x**2 - x + 1/6, x**3 - 3*x**2/2 + 3*x/5 - 1/20, x**4 - 2*x**3 + 9*x**2/7 - 2*x/7 + 1/70]
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal
Ortogonal


### **As séries de Fourier**

**Exercício 54.5**

In [59]:
## Função que retorna o produto interno entre duas funções:
def num_inner_product(f, g, var, a, b):
    num_inner_product = Integral(f*g, (var, a, b)).evalf()
    return num_inner_product

## Função que retorna a projeção ortogonal de uma função f sobre outra função g entre duas funções:
def num_orthogonal_projection(f, g, var, a, b):
    num_orthogonal_projection = (num_inner_product(f, g, var, a, b)/(num_inner_product(g, g, var, a, b)))*g
    return num_orthogonal_projection

In [60]:
## Comparação entre inner_product e num_inner_product:
print(f"inner_product: {inner_product(cos(x), sin(x), x, -2, 1)}")
print(f"num_inner_product: {num_inner_product(cos(x), sin(x), x, -2, 1)}")

inner_product: -sin(2)**2/2 + sin(1)**2/2
num_inner_product: -0.0593741960791174


**Exercício 54.6**

In [61]:
## Função que dada função f calcula a aproximação de Fourier até o grau 5:
def fourier_drg_5(f):
    f_t = 0
    tolerance = 1e-5
    for m in range(1, 6):
        # Coeficientes para cosseno e seno
        c_m = (1 / pi) * num_inner_product(f, cos(m * x), x, -pi, pi)
        s_m = (1 / pi) * num_inner_product(f, sin(m * x), x, -pi, pi)
        
        c_m = 0 if abs(c_m) < tolerance else c_m
        s_m = 0 if abs(s_m) < tolerance else s_m

        # Soma os termos à série
        f_t += c_m * cos(m * t) + s_m * sin(m * t)
    
    # Termo constante (a_0 / 2)
    a_0 = (1 / (2 * pi)) * num_inner_product(f, 1, x, -pi, pi)
    a_0 = 0 if abs(a_0) < tolerance else a_0
    f_t += a_0
    f_t = f_t.evalf()

    return f_t

In [62]:
# Testes
print(fourier_drg_5(x))
print(fourier_drg_5(x**2))
print(fourier_drg_5(abs(x)))

2.0*sin(t) - 1.0*sin(2*t) + 0.666666666666667*sin(3*t) - 0.5*sin(4*t) + 0.4*sin(5*t) + 1.77648528529575e-104*cos(t) - 1.77648528529575e-104*cos(2*t) + 8.88242642647873e-105*cos(3*t) - 3.55297057059149e-104*cos(4*t) - 1.77648528529575e-104*cos(5*t)
-5.42140284819258e-109*sin(t) + 1.3878791291373e-106*sin(2*t) + 4.44121321323937e-105*sin(3*t) + 1.77648528529575e-104*sin(4*t) + 4.44121321323937e-105*sin(5*t) - 4.0*cos(t) + 1.0*cos(2*t) - 0.444444444444444*cos(3*t) + 0.25*cos(4*t) - 0.16*cos(5*t) + 3.28986813369645
1.11030330330984e-105*sin(t) + 2.22060660661968e-105*sin(2*t) + 8.88242642647873e-105*sin(3*t) + 8.88242642647873e-105*sin(4*t) - 8.88242642647873e-105*sin(5*t) - 1.27323954473516*cos(t) - 4.92846766282387e-6*cos(2*t) - 0.141475377880271*cos(3*t) - 4.92861588741523e-6*cos(4*t) - 0.0509343902425822*cos(5*t) + 1.57080022691234


**Exercício 54.7**

In [63]:
f = abs(x)
plot(f, xlim=(-3,3), ylim=(-1,4))

     10 |\                                                     /
        | ..                                                 .. 
        |   \                                               /   
        |    \                                             /    
        |     ..                                         ..     
        |       \                                       /       
        |        \                                     /        
        |         ..                                 ..         
        |           \                               /           
        |            \                             /            
      5 |-------------..-------------------------..-------------
        |               \                       /               
        |                \                     /                
        |                 ..                 ..                 
        |                   \               /                   
        |                

In [64]:
def fourier(f, var, L, k):   
    
    a0 = (1/(2*L))*integrate(f, (var, -L, L))
    fourier_series = a0 
    
    for n in range(1, k + 1):
        an = (1/L)*integrate(f*cos((n*pi*var)/L), (var, -L, L))
        bn = (1/L)*integrate(f*sin((n*pi*var)/L), (var, -L, L))
        
        fourier_series += an*cos((n*pi*var)/L) + bn*sin((n*pi*var)/L)
    
    return fourier_series 

In [67]:
# Testes:
k = int(input("Digite o grau (k) para a aproximação de Fourier: "))

f1 = x
f2 = x**2
f3 = exp(x)

fourier_seriesf1 = fourier(f1, x, pi, 10)
fourier_seriesf2 = fourier(f2, x, pi, 10)
fourier_seriesf3 = fourier(f3, x, pi, 10)

plot(fourier_seriesf1, (x, -3, 3), ylim=(-1, 4), title=f'Série de Fourier de {f1} até o grau {k}')
plot(fourier_seriesf2, (x, -3, 3), ylim=(-1, 4), title=f'Série de Fourier de {f2} até o grau {k}')
plot(fourier_seriesf3, (x, -3, 3), ylim=(-1, 4), title=f'Série de Fourier de {f3} até o grau {k}')

    3.4 |                                                     . 
        |                                                    /  
        |                                                   /   
        |                                               ....   .
        |                                              /        
        |                                         .....         
        |                                     . ..              
        |                                   .. .                
        |                               ....                    
        |                              /                        
      0 |-------------------------.....-------------------------
        |                        /                              
        |                    ....                               
        |                . ..                                   
        |              .. .                                     
        |         .....  

In [66]:
# Gráficos
f1 = x
plot(f1, xlim=(-3,3), ylim=(-1,4))
f2 = x**2
plot(f2, xlim=(-3,3), ylim=(-1,4))
f3 = exp(x)
plot(f3, xlim=(-3,3), ylim=(-1,1)) 

     10 |                                                     ..
        |                                                  ...  
        |                                                ..     
        |                                             ...       
        |                                          ...          
        |                                        ..             
        |                                     ...               
        |                                  ...                  
        |                                ..                     
        |                             ...                       
      0 |--------------------------...--------------------------
        |                       ...                             
        |                     ..                                
        |                  ...                                  
        |               ...                                     
        |             .. 